In [1]:
# Pipeline runner for 8videos_mouse7 (runs without editing suns/config.py on disk)
import os, sys, pathlib, runpy

# Repo root (git) and training folder
REPO_ROOT = pathlib.Path("/gpfs/home/bizzin01/nicole/code/SUNS_nicole_git/Shallow-UNet-Neuron-Segmentation_SUNS").resolve()
TRAIN_DIR = REPO_ROOT / "demo" / "train_3_test_1"
assert REPO_ROOT.exists() and TRAIN_DIR.exists(), (REPO_ROOT, TRAIN_DIR)

# Make repo importable
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

# Configure Python output to be unbuffered inside notebook
os.environ.setdefault("PYTHONUNBUFFERED", "1")

# Import config and set active dataset only in this kernel
from suns import config as suns_config
print("Before:", suns_config.ACTIVE_EXP_SET)
suns_config.ACTIVE_EXP_SET = "8videos_mouse7"
print("Now set to:", suns_config.ACTIVE_EXP_SET)
print("Data folder:", suns_config.DATAFOLDER_SETS[suns_config.ACTIVE_EXP_SET])
print("Exp IDs:", suns_config.EXP_ID_SETS[suns_config.ACTIVE_EXP_SET])



importing config
Before: 4video mouse7
Now set to: 8videos_mouse7
Data folder: /gpfs/home/bizzin01/nicole/code/SUNS_nicole/demo/8videos_mouse7
Exp IDs: ['mouse7_773', 'mouse7_774', 'mouse7_775', 'mouse7_776', 'mouse7_777', 'mouse7_778', 'mouse7_779', 'mouse7_780']


In [2]:
# Helper to run a repo script in-process so it sees the modified suns.config
from typing import List, Optional

def run_repo_script(rel_path: str, args: Optional[List[str]] = None):
    script_path = (REPO_ROOT / rel_path).resolve()
    assert script_path.exists(), f"Script not found: {script_path}"
    print(f"\n=== Running: {script_path} ===")
    # Ensure cwd is the script's directory (matches demo shell behavior)
    cwd = str(script_path.parent)
    # Simulate argv for scripts that may read it
    sys_argv_backup = sys.argv[:]
    try:
        sys.argv = [str(script_path)] + (args or [])
        # run_name='__main__' executes the script as if from CLI
        result = runpy.run_path(str(script_path), run_name='__main__')
        return result
    finally:
        sys.argv = sys_argv_backup
    
print("Helper ready.")



Helper ready.


In [3]:
# 1) Generate sparse GT masks for 8videos_mouse7
run_repo_script("demo/train_3_test_1/generate_sparse_GT.py")
print("Done sparse GT generation.")




=== Running: /gpfs/data/shohamlab/nicole/code/SUNS_nicole_git/Shallow-UNet-Neuron-Segmentation_SUNS/demo/train_3_test_1/generate_sparse_GT.py ===
importing config
FinalMasks_mouse7_773.mat
FinalMasks_mouse7_775.mat
FinalMasks_mouse7_776.mat
FinalMasks_mouse7_774.mat
Done sparse GT generation.


In [4]:
# 2) Train CNN with params (streams logs here)
run_repo_script("demo/train_3_test_1/demo_train_CNN_params.py")
print("Training stage completed.")




=== Running: /gpfs/data/shohamlab/nicole/code/SUNS_nicole_git/Shallow-UNet-Neuron-Segmentation_SUNS/demo/train_3_test_1/demo_train_CNN_params.py ===
importing config
TensorFlow version: 2.12.1
Visible GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
  - /physical_device:GPU:0 | Tesla V100-SXM2-16GB | CC=(7, 0)
GPU sanity check: OK (matmul on /GPU:0)
{'list_minArea': [5.0, 6.0, 6.0, 7.0, 8.0, 9.0, 10.0, 10.0, 11.0, 12.0, 13.0], 'list_avgArea': [28.0], 'list_thresh_pmap': [130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230], 'thresh_COM0': 0.798, 'list_thresh_COM': [1.596, 1.995, 2.394, 2.793, 3.192], 'list_thresh_IOU': [0.5], 'thresh_mask': 0.5, 'list_cons': [1, 2, 3, 4, 5, 6, 7]}
Initialization: 9.059906005859375e-05 s
data loading: 163.85165810585022 s
temporal filtering: 1.3269290924072266 s
median computation: 3.971613883972168 s
normalization: 0.0981597900390625 s
total per frame: 3.373090624809265 ms
Network_input saving: 0.23946833610534668 s
[DEBUG] F

: 

: 

: 

In [ ]:
# 3) Run SUNS batch inference
run_repo_script("demo/train_3_test_1/demo_test_batch.py")
print("Batch inference completed.")



In [ ]:
# 4) Run SUNS online
run_repo_script("demo/train_3_test_1/demo_test_online.py")
print("Online run completed.")



In [ ]:
# 5) Run SUNS online with tracking
run_repo_script("demo/train_3_test_1/demo_test_online_track.py")
print("Online tracking run completed.")

